In [1]:
import os
import pickle
import numpy as np
from scipy.stats import kendalltau
import csv

In [2]:
# Check if running on Colab
try:
  import google.colab
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  from google.colab import drive
  # Connect to Google drive where the training data is located
  drive.mount("/content/gdrive")
  work_dir = "/content/gdrive/My Drive/Colab Notebooks/DL-Project-2024-Experiments/SUBMISSION"
  os.chdir(work_dir)
  print(f"Connected to Google drive, setting working directory to '{work_dir}'")

Mounted at /content/gdrive
Connected to Google drive, setting working directory to '/content/gdrive/My Drive/Colab Notebooks/DL-Project-2024-Experiments/SUBMISSION'


In [3]:
# Create results folder if it does not exist yet
results_folder_name = 'kendalls_tau_results'
if not os.path.exists(results_folder_name):
    os.makedirs(results_folder_name)

In [4]:
# look for the first available baseline pickle file
baseline_file_name_start="task_accuracies_data_e2e_ep_50_ex_"
baseline_file_name_end="_b_0.0_gl_0.0_mpe_0.0_gll_0.0_glh_0.0_mpeh_False.pkl"
folder="raw_result_data"
tau_1_list=[]
tau_2_list=[]
p_1_value_list=[]
p_2_value_list=[]
for filename in os.listdir(folder):
  if filename.startswith(baseline_file_name_start) and filename.endswith(baseline_file_name_end):
    file_path = os.path.join(folder, filename)
    loaded_results = None
    with open(file_path, 'rb') as f:
      loaded_results = pickle.load(f)
      learning_speeds=loaded_results["learning_speeds"]
      sensitivities=loaded_results["sensitivities"]

      learning_speeds_ranking=np.argsort(learning_speeds)
      sensitivities_ranking=np.argsort(sensitivities)

      # standard comparison
      tau_1, p_value_1=kendalltau(learning_speeds_ranking, sensitivities_ranking)
      tau_1_list.append(tau_1)
      p_1_value_list.append(p_value_1)
      # opposite comparison
      tau_2, p_value_2=kendalltau(learning_speeds_ranking, sensitivities_ranking[::-1])
      tau_2_list.append(tau_2)
      p_2_value_list.append(p_value_2)

print(tau_1_list)
print(p_1_value_list)
print(tau_2_list)
print(p_2_value_list)

# write CSV file for report
header = ['Direction', 'Kendall\'s Tau', 'p-value']
rows = [
    ['Same', tau_1_list[0], p_1_value_list[0]],
    ['Opposite', tau_2_list[0], p_2_value_list[0]]
]

kendall_file_name=f'{results_folder_name}/kendalls_tau.csv'
with open(kendall_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

print(f"CSV file {kendall_file_name} created successfully.")


[-0.0011045817832713307, 0.004438238329533181, -0.0005805224208968358, 0.0014203896155846232, 0.0008807840313612543, -0.0015441321652866111, -0.0013770790831633263, 0.0020644857794311767, 0.00045466138645545816, -0.004576490259610384]
[0.793355183435978, 0.2925486747454308, 0.8904985871406227, 0.7362289260583682, 0.8345406269086395, 0.7142177337680569, 0.7439854810755657, 0.6244158990900783, 0.914134615982974, 0.2777737548156817]
[-0.002659690387615504, 0.0025597759910396412, 0.00044012640505620217, 0.0013958254330173206, 0.002770229209168366, -0.00425773830953238, 0.004713647745909836, 0.0002378655146205848, 0.0038909844393775744, 0.0022173174926997078]
[0.5281995479516954, 0.5438099744108846, 0.9168695109909689, 0.740624924372486, 0.5111992051360036, 0.3126195742980077, 0.26362616243820924, 0.9550145443802512, 0.3561305212212136, 0.5989950658102683]
CSV file kendalls_tau_results/kendalls_tau.csv created successfully.
